In [1]:
%pip install pyspark delta-spark pandas setuptools


[notice] A new release of pip is available: 23.2.1 -> 25.0
[notice] To update, run: pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [2]:
import pyspark
from delta import configure_spark_with_delta_pip

builder = pyspark.sql.SparkSession.builder.appName("aig") \
    .config("spark.sql.extensions", "io.delta.sql.DeltaSparkSessionExtension") \
    .config("spark.sql.catalog.spark_catalog", "org.apache.spark.sql.delta.catalog.DeltaCatalog")

ss = configure_spark_with_delta_pip(builder).getOrCreate()


25/02/03 19:34:44 WARN Utils: Your hostname, codespaces-f6f13d resolves to a loopback address: 127.0.0.1; using 10.0.1.201 instead (on interface eth0)
25/02/03 19:34:44 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address


:: loading settings :: url = jar:file:/workspaces/hse-hw-2025-1/.venv/lib/python3.12/site-packages/pyspark/jars/ivy-2.5.1.jar!/org/apache/ivy/core/settings/ivysettings.xml


Ivy Default Cache set to: /home/codespace/.ivy2/cache
The jars for the packages stored in: /home/codespace/.ivy2/jars
io.delta#delta-spark_2.12 added as a dependency
:: resolving dependencies :: org.apache.spark#spark-submit-parent-2156aa21-d8eb-4898-9be2-b1426965ea2b;1.0
	confs: [default]
	found io.delta#delta-spark_2.12;3.3.0 in central
	found io.delta#delta-storage;3.3.0 in central
	found org.antlr#antlr4-runtime;4.9.3 in central
:: resolution report :: resolve 221ms :: artifacts dl 6ms
	:: modules in use:
	io.delta#delta-spark_2.12;3.3.0 from central in [default]
	io.delta#delta-storage;3.3.0 from central in [default]
	org.antlr#antlr4-runtime;4.9.3 from central in [default]
	---------------------------------------------------------------------
	|                  |            modules            ||   artifacts   |
	|       conf       | number| search|dwnlded|evicted|| number|dwnlded|
	---------------------------------------------------------------------
	|      default     |   3   

In [3]:

ss.sql("SELECT 1 AS x").write.format("delta").mode("overwrite").saveAsTable('test')

25/02/03 19:34:56 WARN SparkStringUtils: Truncated the string representation of a plan since it was too large. This behavior can be adjusted by setting 'spark.sql.debug.maxToStringFields'.
25/02/03 19:34:56 WARN GarbageCollectionMetrics: To enable non-built-in garbage collector(s) List(G1 Concurrent GC), users should configure it(them) to spark.eventLog.gcMetrics.youngGenerationGarbageCollectors or spark.eventLog.gcMetrics.oldGenerationGarbageCollectors


In [4]:

ss.sql("DESCRIBE HISTORY test").toPandas()

,version,timestamp,userId,userName,operation,operationParameters,job,notebook,clusterId,readVersion,isolationLevel,isBlindAppend,operationMetrics,userMetadata,engineInfo
0,8,2025-02-03 19:35:07.766,None,None,CREATE OR REPLACE TABLE AS SELECT,"{'partitionBy': '[]', 'description': None, 'pr...",None,None,None,7.0,Serializable,False,"{'numOutputRows': '1', 'numOutputBytes': '449'...",None,Apache-Spark/3.5.4 Delta-Lake/3.3.0
1,7,2025-02-03 19:32:28.318,None,None,CREATE OR REPLACE TABLE AS SELECT,"{'partitionBy': '[]', 'description': None, 'pr...",None,None,None,6.0,Serializable,False,"{'numOutputRows': '1', 'numOutputBytes': '449'...",None,Apache-Spark/3.5.4 Delta-Lake/3.3.0
2,6,2025-02-03 19:31:48.126,None,None,CREATE OR REPLACE TABLE AS SELECT,"{'partitionBy': '[]', 'description': None, 'pr...",None,None,None,5.0,Serializable,False,"{'numOutputRows': '1', 'numOutputBytes': '449'...",None,Apache-Spark/3.5.4 Delta-Lake/3.3.0
3,5,2025-02-03 19:27:51.154,None,None,CREATE OR REPLACE TABLE AS SELECT,"{'partitionBy': '[]', 'description': None, 'pr...",None,None,None,4.0,Serializable,False,"{'numOutputRows': '1', 'numOutputBytes': '449'...",None,Apache-Spark/3.5.4 Delta-Lake/3.3.0
4,4,2025-02-03 19:26:08.526,None,None,CREATE OR REPLACE TABLE AS SELECT,"{'partitionBy': '[]', 'description': None, 'pr...",None,None,None,3.0,Serializable,False,"{'numOutputRows': '1', 'numOutputBytes': '449'...",None,Apache-Spark/3.5.4 Delta-Lake/3.3.0
5,3,2025-02-03 19:25:53.266,None,None,CREATE OR REPLACE TABLE AS SELECT,"{'partitionBy': '[]', 'description': None, 'pr...",None,None,None,2.0,Serializable,False,"{'numOutputRows': '1', 'numOutputBytes': '449'...",None,Apache-Spark/3.5.4 Delta-Lake/3.3.0
6,2,2025-02-03 19:24:23.770,None,None,CREATE OR REPLACE TABLE AS SELECT,"{'partitionBy': '[]', 'description': None, 'pr...",None,None,None,1.0,Serializable,False,"{'numOutputRows': '1', 'numOutputBytes': '449'...",None,Apache-Spark/3.5.4 Delta-Lake/3.3.0
7,1,2025-02-03 19:22:34.862,None,None,CREATE OR REPLACE TABLE AS SELECT,"{'partitionBy': '[]', 'description': None, 'pr...",None,None,None,0.0,Serializable,False,"{'numOutputRows': '1', 'numOutputBytes': '449'...",None,Apache-Spark/3.5.4 Delta-Lake/3.3.0
8,0,2025-02-03 19:20:30.402,None,None,CREATE TABLE AS SELECT,"{'partitionBy': '[]', 'description': None, 'pr...",None,None,None,NaN,Serializable,True,"{'numOutputRows': '1', 'numOutputBytes': '449'...",None,Apache-Spark/3.5.4 Delta-Lake/3.3.0


In [13]:
ss.sql("""
CREATE OR REPLACE TABLE test_cluster 
USING delta CLUSTER BY (x)
AS SELECT * FROM test
""").collect()

25/02/03 19:38:37 WARN OptimisticTransaction: Change in the table id detected in txn. Table id for txn on table at file:/workspaces/hse-hw-2025-1/spark-warehouse/test_cluster was 2cb5688e-ed73-467e-9e21-84c350077631 when the txn was created and is now changed to e8c64d0f-e3fe-4440-bd19-2a337aaa0814.
25/02/03 19:38:37 WARN DeltaLog: Change in the table id detected while updating snapshot. 
Previous snapshot = Snapshot(path=file:/workspaces/hse-hw-2025-1/spark-warehouse/test_cluster/_delta_log, version=1, metadata=Metadata(2cb5688e-ed73-467e-9e21-84c350077631,null,null,Format(parquet,Map()),{"type":"struct","fields":[{"name":"x","type":"integer","nullable":true,"metadata":{}}]},ArrayBuffer(),Map(),Some(1738611481681)), logSegment=LogSegment(file:/workspaces/hse-hw-2025-1/spark-warehouse/test_cluster/_delta_log,1,List(DeprecatedRawLocalFileStatus{path=file:/workspaces/hse-hw-2025-1/spark-warehouse/test_cluster/_delta_log/00000000000000000000.json; isDirectory=false; length=1051; replicati

[]

In [15]:
ss.sql("SELECT * FROM test_cluster WHERE x = '1'").toPandas()

,x
0,1
